In [ ]:
import unicodedata
import csv
import random
from openai import OpenAI
from sklearn.metrics import mean_absolute_error
import numpy as np

sample_num = 100
example_num = 4
content_truncate = 150

In [ ]:
csv_name = 'bace.csv'
data = []
with open(csv_name, 'r') as file:
    reader = csv.reader(file)
    headers = next(reader)  
    for row in reader:
        data.append(tuple(row))
data_set = set(data)

data_sample = random.sample(data, sample_num)
data_sample_set = set(data_sample)
example_set = data_set - data_sample_set
sample_example = random.sample(list(example_set), example_num * sample_num)

sample_example = np.array(sample_example).reshape(sample_num, example_num, len(headers))

print(data_sample)

In [ ]:


# mol_id_index = headers.index('mol_id')
# smiles_index = headers.index('smiles')
# A_index = headers.index('A')
# B_index = headers.index('B')
# C_index = headers.index('C')
# description_index = headers.index('Description')
# mu_index = headers.index('mu')
# alpha_index = headers.index('alpha')
bace_index = headers.index('')

contents = []
bace_true = []
for data_i in range(len(data_sample)):
    data = data_sample[data_i]
    content = "You are an expert chemist with a comprehensive understanding of molecular structures and chemistry. Your task is to predict the property of molecule using your experienced chemical property prediction knowledge.\nPlease strictly follow the format, no other information can be provided. Given the information of a molecule, predict the molecular properties of a given chemical compound based on its structure, by analyzing whether it can inhibit(Yes) the Beta-site Amyloid Precursor Protein Cleaving Enzyme 1 (BACE1) or cannot inhibit(No) BACE1. Consider factors such as molecular weight, atom count, bond types, and functional groups in order to assess the compound's drug-likeness and its potential to serve as an effective therapeutic agent for Alzheimer's disease,please answer with only Yes or No. \n"
    content += "Here are some examples:\n"
    example = sample_example[data_i]

    for example_i in range(example_num):
        example_data = example[example_i]
        content += "Example " + str(example_i + 1) + ":\n"
        content += "Molecule ID: " + example_data[mol_id_index] + "\n"
        content += "SMILES string: " + example_data[smiles_index] + "\n"
        content += "Coordinates: " + example_data[A_index] + ", " + example_data[B_index] + ", " + example_data[C_index] + "\n"
        content += "Description: " + example_data[description_index][:content_truncate] + ".\n"
        content += "Dipole Moment (mu): " + example_data[mu_index] + "\n"
        content += "Isotropic Polarizability (alpha): " + example_data[alpha_index] + "\n\n"
    
    content += "Molecule ID: " + data[mol_id_index] + "\n"
    content += "SMILES string: " + data[smiles_index] + "\n"
    content += "Coordinates: " + data[A_index] + ", " + data[B_index] + ", " + data[C_index] + "\n"
    content += "Description: " + data[description_index][:content_truncate] + ".\n\n"
    content += "Provide your answer of the molecule with Yes or No and avoid any additional information."
    contents.append(content)
    bace_true.append(data[bace_index])

print(contents[0])

In [ ]:
client = OpenAI(
    api_key="your api key"
)

answer_contents = []
for content in contents:
    completion = client.chat.completions.create(
        messages=[
        {"role": "system", "content": "You are an experienced chemist."},
        {"role": "user", "content": content}
        ],
        model="gpt-3.5-turbo-1106", #you can change models
    )

    message = completion.choices[0].message
    message_content = unicodedata.normalize('NFKC', message.content)
    print('message_content:', message_content)

    answer_contents.append(message_content)

In [ ]:
# evaluation